In [ ]:
"""

to install sounddevice:
pip install sounddevice --user

requires numpy and scipy

tuner is from github user - michniewicz

"""

In [9]:
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav

#records the audio of the string plucked and plays it back
def record_and_play_audio():
    duration = int(input("How many seconds would you like to record?: ")) 
    myrecording = sd.rec(duration * fs, samplerate=fs, channels=2, dtype='float64')
    print("Recording now! Play your note once.")
    sd.wait()
    print("Recording complete, playing audio...")
    sd.play(myrecording, fs)


fs=44100
try: 
    record_and_play_audio()

    note_min = 40
    note_max = 64
    fsamp = 22050
    frame_size = 2048
    frame_size2 = bytes
    #fft stands for "fast fourier transform", which is a measurement of audio and acoustics
    frames_per_fft = 16

    samples_per_fft = frame_size * frames_per_fft
    freq_step = float(fsamp) / samples_per_fft

    note_names = 'E F F# G G# A A# B C C# D D#'.split()

    def freq_to_number(f): 
        frequency_to_number = (64 + 12 * np.log2(f / 329.63))
        return frequency_to_number
    def number_to_freq(n): 
        number_to_frequency = (329.63 * 2.0**((n - 64) / 12.0))
        return number_to_frequency
    def note_name(n): 
        note_name = (note_names[n % note_min % len(note_names)] + str(int(n / 12 - 1)))
        return note_name

    def note_to_fftbin(n): 
        return number_to_freq(n) / freq_step

    imin = max(0, int(np.floor(note_to_fftbin(note_min - 1))))
    imax = min(samples_per_fft, int(np.ceil(note_to_fftbin(note_max + 1))))

    buffer = np.zeros(samples_per_fft, dtype=np.float32)
    num_frames = 0


    stream = sd.RawInputStream()
    stream.start()

    window = 0.5 * (1 - np.cos(np.linspace(0, 2 * np.pi, samples_per_fft, False)))

    # This prints the initial text
    print("Sampling at", fsamp, "Hz with maximum resolution of", freq_step, "Hz")
    print()


    # As long as we are getting data:
    #if stream.active():

    # Shift the buffer down and new data in
    buffer[:-frame_size] = buffer[frame_size:]
    buffer[-frame_size:] = np.fromstring(stream.read(frame_size2), np.int16)

    # Run the FFT on the windowed buffer
    fft = np.fft.rfft(buffer * window)

    # Get frequency of maximum response in range
    freq = (np.abs(fft[imin:imax]).argmax() + imin) * freq_step

    # Get note number and nearest note
    n = freq_to_number(freq)
    n0 = int(round(n))

    # Console output once we have a full buffer
    num_frames += 1

    if num_frames >= frames_per_fft:
        print('number {:7.2f} freq: {:7.2f} Hz note: {:>3s} {:+.2f}'.format(n, freq, note_name(n0), n - n0))

except ValueError:
    print("Please enter an integer for number of seconds!")

How many seconds would you like to record?: 5
Recording now! Play your note.
Recording complete, playing audio...


TypeError: must be str, not int